In [2]:
### parameters
place = 'tel_aviv'
feature = 'sidewalk_width'


In [3]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import MultiLineString, LineString, Point


from tqdm import tqdm
import numpy as np
import os
from pathlib import Path
import warnings
warnings.filterwarnings(action='ignore')
crs_prj = 'EPSG:2039'




# Get the current working directory (e.g., the folder you're running from)
cwd = Path().resolve()

# Get the parent directory
parent_folder = f'{cwd.parent}/places/{place}'
data_folder = f'{parent_folder}/shp'
os.makedirs(f'{parent_folder}',exist_ok=True)
os.makedirs(f'{parent_folder}/shp',exist_ok=True)
os.makedirs(f'{parent_folder}/shp/{feature}',exist_ok=True)
detail_folder = f'{data_folder}/{feature}'

In [4]:
streets = gpd.read_file(f'{data_folder}/streets.shp')
street_edges = streets.copy()
no_sidewalks = gpd.read_file(f'{detail_folder}/no_sidewalks.shp')
sidewalks = gpd.read_file(f'{detail_folder}/sidewalks.shp')

In [5]:


def create_perpendicular_line(point, direction_vector, length=25, side='left'):
    dx, dy = direction_vector
    mag = np.hypot(dx, dy)
    if mag == 0:
        return None
    dx /= mag
    dy /= mag

    # Rotate ±90° to get perpendicular direction
    perp_dx, perp_dy = (-dy, dx) if side == 'left' else (dy, -dx)
    end_point = Point(point.x + perp_dx * length, point.y + perp_dy * length)
    return LineString([point, end_point])

def get_point_and_direction(line, fraction):
    point = line.interpolate(fraction, normalized=True)
    # Estimate direction using a small delta on the line
    delta = 0.001
    p1 = line.interpolate(max(fraction - delta, 0), normalized=True)
    p2 = line.interpolate(min(fraction + delta, 1), normalized=True)
    direction = np.array([p2.x - p1.x, p2.y - p1.y])
    return point, direction

# Generate side lines
side_lines = []

for idx, row in street_edges.iterrows():
    geom = row.geometry

    if not isinstance(geom, LineString) or geom.length == 0:
        print(f'{idx} is not a valid street edge')
        continue

    # Get 1/3 and 2/3 points and direction vectors
    for loc,fraction in {'start' :1/3,'middle':0.5,'end': 2/3}.items():
        point, direction = get_point_and_direction(geom, fraction)
        for side in ['left', 'right']:
            line = create_perpendicular_line(point, direction, side=side)
            if line:
                side_lines.append({
                    'oidrechov': row['oidrechov'],
                    'side':f'{loc}_{side}',
                    'geometry': line,
                })

# Convert to GeoDataFrame and save
side_lines_gdf = GeoDataFrame(side_lines, crs=street_edges.crs)
side_lines_gdf.to_file(f'{detail_folder}/side_lines_gdf0.shp', driver='ESRI Shapefile')

# Ensure each perpendicular line has a unique index
side_lines_gdf = side_lines_gdf.reset_index(drop=True)

# Spatial join: find which perpendicular lines intersect which street edges
joined = gpd.sjoin(side_lines_gdf, GeoDataFrame(geometry= street_edges.geometry.buffer(0.005), crs=crs_prj), how='left', predicate='intersects')
joined['line_index'] = joined.index

# Count how many street segments each perpendicular line intersects
intersection_counts = (
    joined.groupby('line_index')
    .size()
    .reset_index(name='count')
)
# Merge counts back to original
side_lines_gdf_0= side_lines_gdf.merge(intersection_counts, left_index=True, right_on='line_index', how='left').drop(columns=['line_index'])
side_lines_gdf_0.to_file(f'{detail_folder}/side_lines_gdf1.shp', driver='ESRI Shapefile')



In [6]:


# -------------------------------------------------------------------
# Helper: opposite side label (e.g. "middle_right" -> "middle_left")
# -------------------------------------------------------------------
def opposite_side_label(side_label: str) -> str | None:
    if "right" in side_label:
        return side_label.replace("right", "left")
    if "left" in side_label:
        return side_label.replace("left", "right")
    return None

# -------------------------------------------------------------------
# Helper: choose closest / farthest intersection for a line
# -------------------------------------------------------------------
def choose_intersection(group, sidewalks, start_point: Point, mode: str = "farthest"):
    """
    Select an intersection from `group` based on distance from `start_point`.

    Parameters
    ----------
    group : pandas.DataFrame
        Subset of `intersections` for a single line_id.
        Must contain column 'index_right'.
    sidewalks : GeoDataFrame
        Sidewalk geometries indexed so that sidewalks.loc[index_right] works.
    start_point : shapely.geometry.Point
        Start point of the side line geometry.
    mode : {'closest', 'farthest'}
        Whether to choose the closest or farthest sidewalk.

    Returns
    -------
    index_right value (scalar) or np.nan if group is empty.
    """
    if group.empty:
        return np.nan

    group = group.copy()

    # Fetch original sidewalk geometries using index_right
    group['sidewalk_geom'] = group['index_right'].apply(
        lambda idx: sidewalks.loc[idx, 'geometry']
    )
    # Compute distance from start point to each sidewalk
    group['dist'] = group['sidewalk_geom'].apply(
        lambda g: g.distance(start_point)
    )

    ascending = (mode == "closest")
    chosen = group.sort_values('dist', ascending=ascending).iloc[0]
    return chosen['index_right']


def compute_intersection_point(res,line_id):
    sidewalk_geom =  sidewalks.loc[res, 'geometry']
    line = side_lines_gdf.loc[line_id].geometry

    return line.intersection(sidewalk_geom)

# ============================================================
# Main logic
# ============================================================

# Step 0: precompute start point for each side line (efficiency)
side_lines_gdf = side_lines_gdf_0.copy()
side_lines_gdf['start_point'] = side_lines_gdf.geometry.apply(
    lambda geom: Point(geom.coords[0])
)

# Get all unique side labels from the 'side' column
side_keys = side_lines_gdf['side'].unique()

# Step 1: Spatial join between side lines and sidewalks
from shapely.ops import nearest_points

# Spatial join: which sidewalk each perpendicular line intersects
intersections = gpd.sjoin(
    side_lines_gdf,
    sidewalks,
    how='left'
).drop(columns=['Layer'])


# Step 2: Group intersections by line index
grouped = intersections.groupby(intersections.index)

# Step 3: Initialize results dictionary
side_line_results = {}

# Step 4: Process each side line
for line_id, group in grouped:
    line_row = side_lines_gdf.loc[line_id]
    side_label = line_row['side']
    oid = line_row['oidrechov']
    start_point = line_row['start_point']

    # Default result
    res = -1
    inter = None
    if len(group) == 1:
        # Single intersection – check if its index_right is NaN
        idx = group.iloc[0]['index_right']
        if not pd.isna(idx):
            # Simple case: one valid intersection
            res = idx
            inter = compute_intersection_point(res,line_id)
        else:
            # idx is NaN → fallback to opposite side
            opp_side = opposite_side_label(side_label)
            if opp_side is not None:
                # Find opposite side line with same oidrechov
                opp_match = side_lines_gdf[
                    (side_lines_gdf['oidrechov'] == oid) &
                    (side_lines_gdf['side'] == opp_side)
                ]

                if not opp_match.empty:
                    opp_line_id = opp_match.index[0]

                    # Only proceed if we have intersections for opposite line
                    if opp_line_id in grouped.groups:
                        opp_group = grouped.get_group(opp_line_id)

                        # Based on your description:
                        #   - If len(opp_group) == 1 → do nothing (keep res = 0)
                        #   - If len(opp_group) > 1 → choose the CLOSEST one
                        if len(opp_group) > 1:
                            opp_line_row = side_lines_gdf.loc[opp_line_id]
                            opp_start = opp_line_row['start_point']
                            res = choose_intersection(
                                opp_group,
                                sidewalks,
                                opp_start,
                                mode="closest"
                            )
                            inter = compute_intersection_point(res,opp_line_id)
                        # else: len == 1 → do nothing, res stays 0

    else:
        # Multiple intersections on this side – choose the FARTHEST from the start
        res = choose_intersection(
            group,
            sidewalks,
            start_point,
            mode="farthest"
        )
        inter = compute_intersection_point(res,line_id)

    # Initialize entry if oid not in results
    if oid not in side_line_results:
        side_line_results[oid] = {key: None for key in side_keys}
        side_line_results[oid]['geometry'] = street_edges.loc[
            street_edges['oidrechov'] == oid, 'geometry'
        ].iloc[0]

    # Store result (0 if res is None / NaN)
    side_line_results[oid][side_label] = float(res) if res is not None and not pd.isna(res) else -1
    side_lines_gdf.loc[line_id,'index_right'] = res
    side_lines_gdf.loc[line_id,'intersection_point'] =  inter

# Step 5: Convert results to GeoDataFrame
records = []
for oid, values in side_line_results.items():
    record = {'oidrechov': oid, 'geometry': values['geometry']}
    for key in side_keys:
        record[key] = values.get(key)
    records.append(record)

gdf = GeoDataFrame(records, crs=crs_prj)
gdf.to_file(f'{detail_folder}/streets_sidewalks_mid.shp', driver='ESRI Shapefile')

In [ ]:
# TEST - START

In [204]:

side_lines_gdf = side_lines_gdf_0.copy()
side_lines_gdf['start_point'] = side_lines_gdf.geometry.apply(
    lambda geom: Point(geom.coords[0])
)

# Get all unique side labels from the 'side' column
side_keys = side_lines_gdf['side'].unique()

# Step 1: Spatial join between side lines and sidewalks
from shapely.ops import nearest_points

# Spatial join: which sidewalk each perpendicular line intersects
intersections = gpd.sjoin(
    side_lines_gdf,
    sidewalks,
    how='left'
).drop(columns=['Layer'])
line_id = 14080
line_row = side_lines_gdf.loc[line_id]
group= grouped.get_group(line_id)
group
# Step 0: precompute start point for each side line (efficiency)

,oidrechov,side,geometry,count,start_point,index_right
14080,1137.0,end_left,"LINESTRING (179875.616 664240.792, 179900.613 ...",1,POINT (179875.616 664240.792),23241.0


In [198]:
side_label = line_row['side']
oid = line_row['oidrechov']
start_point = line_row['start_point']

oidrechov                                                         110.0
side                                                         start_left
geometry              LINESTRING (180643.2573333333 661325.071333333...
count                                                                 1
start_point                 POINT (180643.2573333333 661325.0713333334)
index_right                                                        -1.0
intersection_point                                                 None
Name: 2346, dtype: object

In [178]:
side_lines_gdf[side_lines_gdf['oidrechov']==1137]

,oidrechov,side,geometry,count,start_point,index_right,intersection_point
14076,1137.0,start_left,"LINESTRING (179876.26 664283.96, 179901.258 66...",1,POINT (179876.26 664283.96),23392.0,POINT (179875.77167206872 664283.9675411737)
14077,1137.0,start_right,"LINESTRING (179876.26 664283.96, 179851.263 66...",1,POINT (179876.26 664283.96),23387.0,POINT (179869.68279923932 664284.0584698643)
14078,1137.0,middle_left,"LINESTRING (179875.938 664262.376, 179900.935 ...",1,POINT (179875.938 664262.376),23392.0,POINT (179875.47006374324 664262.3829207476)
14079,1137.0,middle_right,"LINESTRING (179875.938 664262.376, 179850.941 ...",1,POINT (179875.938 664262.376),23387.0,POINT (179869.41105533353 664262.4734034555)
14080,1137.0,end_left,"LINESTRING (179875.616 664240.792, 179900.613 ...",1,POINT (179875.616 664240.792),23241.0,POINT (179890.98336525337 664240.5621270455)
14081,1137.0,end_right,"LINESTRING (179875.616 664240.792, 179850.618 ...",1,POINT (179875.616 664240.792),23387.0,POINT (179869.09119528998 664240.8890555929)


In [142]:
intersections

,oidrechov,side,geometry,count,start_point,index_right,intersection_point
0,1.0,start_left,"LINESTRING (184332.382 668569.389, 184344.027 ...",1,POINT (184332.382 668569.389),9159.0,POINT (184333.38 668571.286)
1,1.0,start_right,"LINESTRING (184332.382 668569.389, 184320.737 ...",1,POINT (184332.382 668569.389),9229.0,POINT (184328.616 668562.235)
2,1.0,middle_left,"LINESTRING (184337.22 668566.842, 184348.866 6...",1,POINT (184337.22 668566.842),9159.0,POINT (184337.838 668568.014)
3,1.0,middle_right,"LINESTRING (184337.22 668566.842, 184325.575 6...",1,POINT (184337.22 668566.842),9229.0,POINT (184333.076 668558.969)
4,1.0,end_left,"LINESTRING (184342.059 668564.295, 184353.704 ...",1,POINT (184342.059 668564.295),9159.0,POINT (184342.682 668565.479)
...,...,...,...,...,...,...,...
52474,9630.0,end_left,"LINESTRING (178868.016 665060.666, 178843.72 6...",1,POINT (178868.016 665060.666),12633.0,POINT (178847.331 665055.653)
52489,9639.0,start_right,"LINESTRING (178804.942 665033.839, 178828.033 ...",1,POINT (178804.942 665033.839),11270.0,POINT (178810.196 665036.019)
52495,9642.0,start_right,"LINESTRING (178830.175 665012.141, 178834.543 ...",1,POINT (178830.175 665012.141),12516.0,POINT (178831.145 665017.608)
52503,9640.0,middle_right,"LINESTRING (178813.567 665071.516, 178832.081 ...",1,POINT (178813.567 665071.516),11324.0,POINT (178817.9 665067.583)


In [ ]:
# Test - Start

In [141]:
gdf[gdf['oidrechov']==1137]


,oidrechov,geometry,start_left,start_right,middle_left,middle_right,end_left,end_right
2346,1137.0,"LINESTRING (179879.525 664326.904, 179876.627 ...",23392.0,23387.0,23392.0,23387.0,23241.0,23387.0


In [139]:
final_intersections[final_intersections['oidrechov']==1137]


,oidrechov,side,geometry,count,start_point,index_right,intersection_point
7066,1137.0,end_left,"LINESTRING (179875.616 664240.792, 179900.613 ...",1,POINT (179875.61566221697 664240.7916219215),23241.0,POINT (179890.98336525337 664240.5621270455)
21296,1137.0,end_right,"LINESTRING (179875.616 664240.792, 179850.618 ...",1,POINT (179875.61566221697 664240.7916219215),23392.0,POINT (179875.11853647546 664240.7990457902)
21297,1137.0,middle_right,"LINESTRING (179875.938 664262.376, 179850.941 ...",1,POINT (179875.93799332544 664262.3759328822),23392.0,POINT (179875.47006374324 664262.3829207476)
21298,1137.0,start_right,"LINESTRING (179876.26 664283.96, 179851.263 66...",1,POINT (179876.2603244339 664283.9602438428),23392.0,POINT (179875.77167206872 664283.9675411737)


In [137]:
final_intersections

,oidrechov,side,geometry,count,start_point,index_right,intersection_point
0,1.0,start_left,"LINESTRING (184332.382 668569.389, 184344.027 ...",1,POINT (184332.382 668569.389),9159.0,POINT (184333.38041888748 668571.2856822867)
1,1.0,start_right,"LINESTRING (184332.382 668569.389, 184320.737 ...",1,POINT (184332.382 668569.389),9229.0,POINT (184328.61591256654 668562.2346167896)
2,1.0,middle_left,"LINESTRING (184337.22 668566.842, 184348.866 6...",1,POINT (184337.2205 668566.842),9159.0,POINT (184337.83753713127 668568.014176741)
3,1.0,middle_right,"LINESTRING (184337.22 668566.842, 184325.575 6...",1,POINT (184337.2205 668566.842),9229.0,POINT (184333.07621016508 668558.9691510148)
4,1.0,end_left,"LINESTRING (184342.059 668564.295, 184353.704 ...",1,POINT (184342.05899999998 668564.295),9159.0,POINT (184342.68241396494 668565.4792907216)
...,...,...,...,...,...,...,...
30221,9659.0,end_right,"LINESTRING (181300.482 671793.997, 181322.108 ...",1,POINT (181300.48218623607 671793.9971006192),1712.0,POINT (181300.84954140405 671793.7840331851)
30222,9659.0,middle_left,"LINESTRING (181231.362 671695.879, 181210.93 6...",1,POINT (181231.36235930314 671695.8792757468),1715.0,POINT (181226.982142084 671698.967363428)
30223,9659.0,middle_right,"LINESTRING (181231.362 671695.879, 181251.795 ...",1,POINT (181231.36235930314 671695.8792757468),1712.0,POINT (181231.5994945906 671695.7120934991)
30224,9659.0,start_left,"LINESTRING (181163.493 671596.918, 181142.343 ...",1,POINT (181163.49254872673 671596.917777964),1712.0,POINT (181163.41632473358 671596.9658236383)


In [ ]:
# Test - End

In [7]:

# Extend line by 50m and buffer 1m
def extend_and_buffer(line, length=50, buffer_dist=1):
    if not isinstance(line, LineString) or len(line.coords) < 2:
        print('error in the geometry of the line')
        return None
    start, end = line.coords[0], line.coords[-1]
    dx, dy = end[0] - start[0], end[1] - start[1]
    norm = (dx**2 + dy**2) ** 0.5
    if norm == 0:
        return None
    dx /= norm
    dy /= norm
    new_end = (start[0] + dx * length, start[1] + dy * length)
    extended = LineString([start, new_end])
    return extended.buffer(buffer_dist)

# Apply to all perpendicular lines
buffered_perp_lines = side_lines_gdf.copy()
buffered_perp_lines['geometry'] = buffered_perp_lines['geometry'].apply(extend_and_buffer)
buffered_perp_lines = buffered_perp_lines.dropna(subset=['geometry']).drop(columns =['start_point','index_right'])
buffered_perp_lines.drop(columns =['intersection_point']).to_file(f'{detail_folder}/extended_line.shp', driver='ESRI Shapefile')
intersections_1 = gpd.sjoin(buffered_perp_lines, no_sidewalks)

intersections_1['no_sidewalk_geom'] = intersections_1['index_right'].apply(lambda idx: no_sidewalks.loc[idx, 'geometry'])


side_fields = side_keys
side_fields_w = [side + '_w' for side in side_keys ]

gdf_widths = gdf.copy()

for side in side_fields:
    gdf_widths[side_fields_w ] = -1.0  # init with -1

# Pre-index: {(oid, side) → line geometry}
line_geom_lookup = {
    (row['oidrechov'], row['side']): row['intersection_point']
    for idx, row in intersections_1.iterrows()
}

# Pre-index: {(oid, side) → intersection GeoDataFrame}
intersection_groups = dict(tuple(intersections_1.groupby(['oidrechov', 'side'])))



# Process efficiently
for idx, row in tqdm(gdf.iterrows(), total=len(gdf), desc="Processing widths"):
    oid = row['oidrechov']
    values = [row[f] for f in side_fields]

    # Fast logic for all 0/-1 cases
    if all((v == 0) or (v == -1) for v in values):
        result_value = 0 if any(v == 0 for v in values) else -1
        for side in side_fields:
            gdf_widths.at[idx, f'{side}_w'] = result_value
        continue

    for side in side_fields:
        sid = row[side]
        if sid in [-1, 0]:
            continue

        # Lookup intersections
        group_key = (oid, side)
        if group_key not in intersection_groups:
            gdf_widths.at[idx, f'{side}_w'] = -1
            continue

        matching = intersection_groups[group_key]

        # Lookup line geometry
        inter_pnt = line_geom_lookup.get(group_key,None)
        if inter_pnt is None or inter_pnt.is_empty:
            gdf_widths.at[idx, f'{side}_w'] = -1
            continue


        # Compute distances only once using vectorized apply
        matching = matching.copy()
        matching['dist_to_start'] = matching['no_sidewalk_geom'].apply(lambda g: g.distance(inter_pnt))
        closest = matching.sort_values('dist_to_start').iloc[0]['no_sidewalk_geom']

        # Distance to sidewalk (true width)
        dist = sidewalks.loc[sid].geometry.distance(closest)
        if dist==0:
            dist =matching.sort_values('dist_to_start').iloc[0]['dist_to_start']
        gdf_widths.at[idx, f'{side}_w'] = dist

gdf_widths.to_file(f'{detail_folder}/gdf_widths.shp', driver='ESRI Shapefile')




Processing widths: 100%|██████████| 8751/8751 [00:46<00:00, 186.74it/s]


In [94]:
# Test - Start`

In [197]:

side_lines_gdf[side_lines_gdf['oidrechov']==1137]

,oidrechov,side,geometry,count,start_point,index_right,intersection_point
14076,1137.0,start_left,"LINESTRING (179876.26 664283.96, 179901.258 66...",1,POINT (179876.26 664283.96),23392.0,POINT (179875.77167206872 664283.9675411737)
14077,1137.0,start_right,"LINESTRING (179876.26 664283.96, 179851.263 66...",1,POINT (179876.26 664283.96),23387.0,POINT (179869.68279923932 664284.0584698643)
14078,1137.0,middle_left,"LINESTRING (179875.938 664262.376, 179900.935 ...",1,POINT (179875.938 664262.376),23392.0,POINT (179875.47006374324 664262.3829207476)
14079,1137.0,middle_right,"LINESTRING (179875.938 664262.376, 179850.941 ...",1,POINT (179875.938 664262.376),23387.0,POINT (179869.41105533353 664262.4734034555)
14080,1137.0,end_left,"LINESTRING (179875.616 664240.792, 179900.613 ...",1,POINT (179875.616 664240.792),23241.0,POINT (179890.98336525337 664240.5621270455)
14081,1137.0,end_right,"LINESTRING (179875.616 664240.792, 179850.618 ...",1,POINT (179875.616 664240.792),23387.0,POINT (179869.09119528998 664240.8890555929)


In [205]:
intersections_1[intersections_1['oidrechov']==3639]

,oidrechov,side,geometry,count,intersection_point,index_right,Layer,no_sidewalk_geom
38358,3639.0,start_left,"POLYGON ((183915.076 668966.227, 183915.026 66...",1,POINT (183942.20245607515 668927.7041789836),61533,2601,LINESTRING (183948.19220205207 668933.79036232...
38358,3639.0,start_left,"POLYGON ((183915.076 668966.227, 183915.026 66...",1,POINT (183942.20245607515 668927.7041789836),70115,2407,LINESTRING (183941.46229332342 668930.34986911...
38358,3639.0,start_left,"POLYGON ((183915.076 668966.227, 183915.026 66...",1,POINT (183942.20245607515 668927.7041789836),70111,2407,LINESTRING (183933.25093309203 668947.11939462...
38358,3639.0,start_left,"POLYGON ((183915.076 668966.227, 183915.026 66...",1,POINT (183942.20245607515 668927.7041789836),61525,2601,LINESTRING (183939.9923377214 668929.249759268...
38358,3639.0,start_left,"POLYGON ((183915.076 668966.227, 183915.026 66...",1,POINT (183942.20245607515 668927.7041789836),61563,2601,LINESTRING (183923.1197404767 668960.888767182...
38358,3639.0,start_left,"POLYGON ((183915.076 668966.227, 183915.026 66...",1,POINT (183942.20245607515 668927.7041789836),77736,2200,LINESTRING (183937.45079593288 668950.40970936...
38359,3639.0,start_right,"POLYGON ((183972.562 668884.377, 183972.612 66...",1,POINT (183945.42202361274 668922.9193810088),77771,2200,LINESTRING (183967.29397904832 668914.54158607...
38360,3639.0,middle_left,"POLYGON ((183928.171 668975.854, 183928.113 66...",1,POINT (183958.91096079675 668940.0000665435),70238,2407,LINESTRING (183959.02177616087 668943.30118012...
38360,3639.0,middle_left,"POLYGON ((183928.171 668975.854, 183928.113 66...",1,POINT (183958.91096079675 668940.0000665435),77730,2200,LINESTRING (183939.43001545232 668962.15991961...
38360,3639.0,middle_left,"POLYGON ((183928.171 668975.854, 183928.113 66...",1,POINT (183958.91096079675 668940.0000665435),61515,2601,LINESTRING (183941.0089291584 668978.140127451...


In [207]:
idx = 6297
row = gdf.loc[idx]
oid = row['oidrechov']
values = [row[f] for f in side_fields]
side= 'start_right'
sid = row[side]
group_key = (oid, side)
matching = intersection_groups[group_key]
inter_pnt = line_geom_lookup.get(group_key,None)
matching['dist_to_start'] = matching['no_sidewalk_geom'].apply(lambda g: g.distance(inter_pnt))
closest = matching.sort_values('dist_to_start').iloc[0]['no_sidewalk_geom']
sidewalks.loc[sid].geometry.distance(closest)

23.603991045934787

In [212]:
?matching.sort_values('dist_to_start').iloc[0]

oidrechov                                                        5381.0
side                                                        start_right
geometry              POLYGON ((183884.0264776736 669813.9751439908,...
count                                                                 1
intersection_point          POINT (183880.7883543107 669860.0055488613)
index_right                                                       68813
Layer                                                              2601
no_sidewalk_geom      LINESTRING (183912.418538617 669829.4232890663...
dist_to_start                                                 31.178143
Name: 37783, dtype: object

In [209]:
sidewalks.loc[sid]

Layer                                                    2404
geometry    LINESTRING (183937.74686905532 669861.26526042...
Name: 8471, dtype: object

In [210]:
matching.sort_values('dist_to_start').iloc[0]

oidrechov                                                        5381.0
side                                                        start_right
geometry              POLYGON ((183884.0264776736 669813.9751439908,...
count                                                                 1
intersection_point          POINT (183880.7883543107 669860.0055488613)
index_right                                                       68813
Layer                                                              2601
no_sidewalk_geom      LINESTRING (183912.418538617 669829.4232890663...
dist_to_start                                                 31.178143
Name: 37783, dtype: object

In [ ]:
        matching = intersection_groups[group_key]

        # Lookup line geometry
        inter_pnt = line_geom_lookup.get(group_key,None)
        if inter_pnt is None or inter_pnt.is_empty:
            gdf_widths.at[idx, f'{side}_w'] = -1
            continue


        # Compute distances only once using vectorized apply
        matching = matching.copy()
        matching['dist_to_start'] = matching['no_sidewalk_geom'].apply(lambda g: g.distance(inter_pnt))
        closest = matching.sort_values('dist_to_start').iloc[0]['no_sidewalk_geom']

        # Distance to sidewalk (true width)
        dist = sidewalks.loc[sid].geometry.distance(closest)
        gdf_widths.at[idx, f'{side}_w'] = dist

In [95]:
# Test - End

Processing widths: 100%|██████████| 8751/8751 [00:24<00:00, 363.06it/s] 


In [8]:


# Define left and right _w fields
left_fields = [f for f in gdf_widths.columns if f.endswith('_left_w')]
right_fields = [f for f in gdf_widths.columns if f.endswith('_right_w')]

def compute_side_final(values):
    values = [v for v in values if v != -1]
    if len(values) == 0:
        return -1
    if len(values) == 1:
        return values[0]
    if len(values) == 2:
        a, b = values
    else:
        # Pick two closest values
        values = sorted(values)
        best_pair = min(((a, b) for i, a in enumerate(values) for b in values[i+1:]), key=lambda x: abs(x[0] - x[1]))
        a, b = best_pair
    if abs(a - b) > 10:
        return min(a, b)
    return (a + b) / 2

# Apply to each row
def compute_final_columns(row):
    left_values = [row[f] for f in left_fields]
    right_values = [row[f] for f in right_fields]
    final_left = compute_side_final(left_values)
    final_right = compute_side_final(right_values)

    # Compute final
    if final_left == -1 and final_right == -1:
        final = -1
    elif final_left == -1:
        final = final_right
    elif final_right == -1:
        final = final_left
    else:
        final = (final_left + final_right) / 2
    return pd.Series({'final_left': final_left, 'final_right': final_right, 'final': final})

# Apply across DataFrame
gdf_widths[['final_left', 'final_right', 'final']] = gdf_widths.apply(compute_final_columns, axis=1)
gdf_widths.to_file(f'{detail_folder}/{feature}.shp',driver='ESRI Shapefile')

In [ ]:
# Test - Start

In [ ]:
# Test - End

In [42]:
### THis code is for selecting sidewalks form my raw data - I don"t need to run it only for the first time:   ###
# -----------------------------------------------------------------------------
# Step 1: Read the 'translation' layer from a File Geodatabase
# -----------------------------------------------------------------------------
# This loads only the 'geometry' and 'Layer' columns from the specified layer
# within the ESRI File Geodatabase located at 'ASC/ASC.gdb'.
gdf = gpd.read_file('ASC/ASC.gdb', layer="translation")[['geometry', 'Layer']]

# -----------------------------------------------------------------------------
# Step 2: Define a function to drop the Z dimension from 3D MultiLineStrings
# -----------------------------------------------------------------------------
# This function takes a MultiLineString Z geometry and converts it to a
# 2D MultiLineString by removing the Z (elevation) component from each point.
# It assumes that the input is either a MultiLineString Z or already 2D.
def drop_z(geom):
    # Check if the geometry has a Z dimension
    if geom.has_z:
        # Rebuild each LineString within the MultiLineString, dropping the Z value
        return MultiLineString([
            LineString([(x, y) for x, y, z in line.coords])  # keep only X and Y
            for line in geom.geoms  # iterate over individual LineStrings
        ])
    else:
        # If the geometry is already 2D, return it unchanged
        return geom

# -----------------------------------------------------------------------------
# Step 3: Apply the drop_z function to each geometry in the GeoDataFrame
# -----------------------------------------------------------------------------
# This transforms the entire geometry column to 2D by removing Z values.
# The result is still a valid GeoDataFrame with MultiLineString geometries.
gdf['geometry'] = gdf['geometry'].apply(drop_z)
gdf = gdf.to_crs(crs_prj)
# -----------------------------------------------------------------------------
# Step 6: Export the translated geometries to a new shapefile
# -----------------------------------------------------------------------------
# The resulting GeoDataFrame is saved as a new shapefile named 'translation2.shp'
# in the 'files' directory. All attributes are preserved.
gdf.to_file(f'{data_folder}/itm_data.shp')

### YOU DONT NEED TO RUN THIS CELL ###
# -----------------------------------------------------------------------------
# Step 1: Filter sidewalk and non-sidewalk features based on the 'Layer' field
# -----------------------------------------------------------------------------
# Sidewalks are identified where 'Layer' == '2404'
# All other features are treated as non-sidewalks
sidewalks = gdf[gdf['Layer'] == '2404']
no_sidewalk = gdf[~(gdf['Layer'] == '2404')]
sidewalks.to_file(f'{data_folder}/sidewalks.shp')
### YOU DONT NEED TO RUN THIS CELL ###
# Optional: Save non-sidewalk features to a shapefile for inspection or use
no_sidewalk.to_file(f'{data_folder}/no_sidewalk.shp')